# MNIST CNN-CBR Twin-System

In [9]:
import pickle
import pandas as pd
import numpy as np
import keras

import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Activation
from keras import backend as K

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

from copy import deepcopy

## Get Data & Check Missing Values

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
oh_y_train = to_categorical(y_train)
oh_y_test = to_categorical(y_test)

In [3]:
# Normalise the data 
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28)
60000 train samples
10000 test samples


In [4]:
img_rows, img_cols = 28, 28

In [5]:
# if K.image_data_format() == 'channels_first':
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
input_shape = (1, img_rows, img_cols)

### Train a Keras CNN:

In [6]:
model = Sequential()

# Cell 1 - 1x28x28
model.add(Conv2D(32, (5, 5), input_shape=(1, 28, 28), padding='same', data_format='channels_first'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

# Cell 2 - 32x4x14
model.add(Conv2D(64, (3, 3), input_shape=(16, 14, 14), padding='same', data_format='channels_first'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

# Output - 64x7x7
model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Dense(50))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation("softmax"))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, oh_y_train,
          batch_size=256,
          epochs=20,
          verbose=1,
          validation_data=(X_test, oh_y_test))

# evaluate the model
scores = model.evaluate(X_train, oh_y_train)
print("Training Set:", "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

scores = model.evaluate(X_test, oh_y_test)
print("Test Set:", "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 156s 3ms/step - loss: 0.4219 - acc: 0.8635 - val_loss: 0.0662 - val_acc: 0.9797
Epoch 2/20
60000/60000 [==============================] - 160s 3ms/step - loss: 0.0889 - acc: 0.9740 - val_loss: 0.0377 - val_acc: 0.9886
Epoch 3/20
60000/60000 [==============================] - 156s 3ms/step - loss: 0.0631 - acc: 0.9812 - val_loss: 0.0350 - val_acc: 0.9881
Epoch 4/20
60000/60000 [==============================] - 157s 3ms/step - loss: 0.0490 - acc: 0.9862 - val_loss: 0.0416 - val_acc: 0.9864
Epoch 5/20
60000/60000 [==============================] - 159s 3ms/step - loss: 0.0412 - acc: 0.9877 - val_loss: 0.0316 - val_acc: 0.9905
Epoch 6/20
60000/60000 [==============================] - 149s 2ms/step - loss: 0.0363 - acc: 0.9896 - val_loss: 0.0264 - val_acc: 0.9917
Epoch 7/20
60000/60000 [==============================] - 145s 2ms/step - loss: 0.0305 - acc: 0.9910 - val_loss: 0.0248 - 

## Train Normal k-NN:
Just for a benchmark

In [7]:
knn_X_train = X_train.reshape(X_train.shape[0], 28*28)
knn_X_test = X_test.reshape(X_test.shape[0], 28*28)

In [10]:
# Train final k-NN 
knn_clf = KNeighborsClassifier(n_neighbors=1, algorithm="brute") 
knn_clf.fit(knn_X_train, y_train)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

In [11]:
# Check the accuracy on this particular split to make sure that it is not too far removed from k-fold.
knn_predictions_test = knn_clf.predict(knn_X_test)
print("k-NN Accuracy Test:", accuracy_score(y_test, knn_predictions_test))

k-NN Accuracy Test: 0.9691


In [12]:
nn_pred = model.predict_classes(X_test)

right = 0
for i in range(len(nn_pred)):
    if knn_predictions_test[i] == nn_pred[i]:
        right += 1
print("Agreement:", right/len(nn_pred))

Agreement: 0.9691


In [13]:
# Check confusion matrix kNN
confusion_matrix(y_test, knn_predictions_test, labels=None, sample_weight=None)

array([[ 973,    1,    1,    0,    0,    1,    3,    1,    0,    0],
       [   0, 1129,    3,    0,    1,    1,    1,    0,    0,    0],
       [   7,    6,  992,    5,    1,    0,    2,   16,    3,    0],
       [   0,    1,    2,  970,    1,   19,    0,    7,    7,    3],
       [   0,    7,    0,    0,  944,    0,    3,    5,    1,   22],
       [   1,    1,    0,   12,    2,  860,    5,    1,    6,    4],
       [   4,    2,    0,    0,    3,    5,  944,    0,    0,    0],
       [   0,   14,    6,    2,    4,    0,    0,  992,    0,   10],
       [   6,    1,    3,   14,    5,   13,    3,    4,  920,    5],
       [   2,    5,    1,    6,   10,    5,    1,   11,    1,  967]])

In [14]:
# Check confusion matrix NN 
confusion_matrix(y_test, model.predict_classes(X_test), labels=None, sample_weight=None)

array([[ 978,    0,    0,    0,    0,    0,    0,    1,    1,    0],
       [   0, 1134,    0,    1,    0,    0,    0,    0,    0,    0],
       [   0,    1, 1030,    0,    0,    0,    0,    1,    0,    0],
       [   0,    0,    0, 1007,    0,    2,    0,    0,    1,    0],
       [   0,    0,    0,    0,  978,    0,    1,    0,    0,    3],
       [   1,    0,    0,    6,    0,  882,    1,    0,    1,    1],
       [   3,    2,    1,    0,    1,    4,  947,    0,    0,    0],
       [   0,    1,    6,    0,    0,    0,    0, 1019,    1,    1],
       [   2,    0,    3,    2,    0,    1,    0,    0,  963,    3],
       [   0,    0,    1,    0,    4,    1,    0,    1,    1, 1001]])

In [15]:
# Save the CBR model to disk
pickle.dump(knn_clf, open('k-nn_model.sav', 'wb'))

In [16]:
# Save Keras Models to disk
model.save("NN.h5")

In [17]:
# Save Dataframes
np.save("X_train", X_train)
np.save("X_test", X_test)
np.save("y_train", y_train)
np.save("y_test", y_test)

np.save("knn_X_train", knn_X_train)
np.save("knn_X_test", knn_X_test)

np.save("oh_y_train", oh_y_train)
np.save("oh_y_test", oh_y_test)